# Real time face mask detector 

In [1]:
#importing dependencies
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

# Learning parameters and specifying directory

In [2]:
#Learning rate
learning_rate=0.0001
#Epochs
epochs=30
#Batch Size
batch_size=32
#Source directory
src_dir=r'C:/Users/PROMIT/Desktop/Face-Mask-Detection/dataset/train'
#Classes
categories=["with_mask","without_mask"]

# Loading the images and labeling

In [3]:
images=[]
classes=[]
for item in categories:
    path=src_dir+'/'+item
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)#preprocessing the image
        images.append(image)
        classes.append(item)
#one hot encoding of target variables       
lb=LabelBinarizer()
classes=lb.fit_transform(classes)
classes=to_categorical(classes)
images=np.array(images,dtype="float32")
classes=np.array(classes)#converting into numpy array

C:\Users\PROMIT\anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


# Splitting train and test dataset

In [4]:

X_train,X_test,Y_train,Y_test=train_test_split(images,classes,test_size=0.2,stratify=classes,random_state=42)

# Image augmentation

In [6]:
#image augmentation
aug=ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.20,
    horizontal_flip=True,
    fill_mode="nearest"
)
    
  


# Transfer Learning

In [7]:
#Loading MobileNetV2 as the base model  with weights that of pre trained imagenet model
baseModel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

In [ ]:
#Adding further layers to the basemodel

In [9]:
#Output of baseModel
face_model=baseModel.output
face_model=AveragePooling2D(pool_size=(7,7))(face_model)#Average pooling
face_model=Flatten(name="flatten")(face_model)#Flattening to obtain 1-D array
#Adding hidden layers
headModel=Dense(64,activation="relu")(face_model)
headModel=Dense(128,activation="relu")(face_model)
headModel=Dense(256,activation="relu")(face_model)

face_model=Dropout(0.5)(face_model)#Dropout to prevent overfitting
face_model=Dense(2,activation="softmax")(face_model)
#Appending input and output
model=Model(inputs=baseModel.input,outputs=face_model)
#Preventing further training in pre-trained baseModel
for layer in baseModel.layers:
    layer.trainable=False
opt=RMSprop(learning_rate=learning_rate,decay=learning_rate/epochs)


# Model Architecture

In [10]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [11]:
#Compiling our model
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

In [12]:
#Training the model
history=model.fit(
    aug.flow(X_train,Y_train,batch_size=batch_size),

    validation_data=(X_train,Y_train),
    epochs=epochs
)

Epoch 1/30
92/92 [==============================] - 190s 2s/step - loss: 0.8768 - accuracy: 0.5746 - val_loss: 0.4095 - val_accuracy: 0.8575
Epoch 2/30
92/92 [==============================] - 171s 2s/step - loss: 0.4664 - accuracy: 0.7839 - val_loss: 0.2209 - val_accuracy: 0.9461
Epoch 3/30
92/92 [==============================] - 182s 2s/step - loss: 0.2910 - accuracy: 0.8766 - val_loss: 0.1454 - val_accuracy: 0.9666
Epoch 4/30
92/92 [==============================] - 189s 2s/step - loss: 0.2237 - accuracy: 0.9145 - val_loss: 0.1119 - val_accuracy: 0.9724
Epoch 5/30
92/92 [==============================] - 190s 2s/step - loss: 0.1796 - accuracy: 0.9315 - val_loss: 0.0907 - val_accuracy: 0.9768
Epoch 6/30
92/92 [==============================] - 167s 2s/step - loss: 0.1460 - accuracy: 0.9465 - val_loss: 0.0778 - val_accuracy: 0.9785
Epoch 7/30
92/92 [==============================] - 159s 2s/step - loss: 0.1255 - accuracy: 0.9543 - val_loss: 0.0688 - val_accuracy: 0.9799
Epoch 8/30
92

In [4]:
maskNet=h_model

In [5]:
import cv2
from os.path import dirname, join

protoPath = join('C:/Users/PROMIT/Desktop/contents/Face-Mask-Detection/face_detector/', 'deploy.prototxt')
modelPath = join('C:/Users/PROMIT/Desktop/contents/DFT/AI_ML/', "res10_300x300_ssd_iter_140000.caffemodel")
#Loading face detection classifier
faceNet=cv2.dnn.readNetFromCaffe(protoPath,modelPath)

In [18]:
#function to detect faces and predict 
def face_det_mask_pred(frame,face_detection_classifier,mask_detector):
    (img_height,img_width)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))
    face_detection_classifier.setInput(blob)
    face_detections=face_detection_classifier.forward()
    
    faces=[]
    locs=[]
    preds=[]
    num_faces=face_detections.shape[2]
    for i in range(num_faces):
        confidence=face_detections[0,0,i,2]
        if confidence>0.5:
            box=face_detections[0,0,i,3:7]*np.array([img_width,img_height,img_width,img_height])
            (startX,startY,endX,endY)=box.astype("int")
            face=frame[startY:endY,startX:endX]
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startX,startY,endX,endY))
    if len(faces)>0:
        faces=np.array(faces,dtype="float32")
        preds=mask_detector.predict(faces)
        
      
    return (locs,preds)

In [28]:
cap=VideoStream(src=0).start()
while True:
  
   
    image=cap.read()
    (locs,preds)=face_det_mask_pred(image,faceNet,h_model)
    for (box,pred) in zip(locs,preds):
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred 
        if mask>withoutMask:
            label="Mask"
            temp_color=(51,255,102)
        else:
            label="Without Mask"
            temp_color=(255,255,0)
            
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,1,temp_color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),(255,0,0),2)
    cv2.namedWindow("output",cv2.WINDOW_NORMAL)
    cv2.imshow("output",image)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

         
cv2.destroyAllWindows()
cap.stop()


In [8]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


In [2]:
from tensorflow.keras.models import load_model

In [3]:
h_model=load_model(r'C:\Users\PROMIT\Desktop\contents\DFT\AI_ML\Dataset\MASK_DETECTOR')